In [ ]:
from XML_utils import *

import ast, jsonlines, regex
import pandas as pd

## Create table with entities from GPT output file

In [2]:
# 1879-1881 # 1935-1938 # 1882-1934 # 1939-1950
files = [f'g:\\My Drive\\Overlijdensdata Curaçao 1831-1950\\GPT output\\batch_kGLF1L7AzcnmT9ksbHandAYp_output.jsonl', f'g:\\My Drive\\Overlijdensdata Curaçao 1831-1950\\GPT output\\batch_jZ32N0g4hRf3WMOB1EAaZBrN_output.jsonl', f'g:\\My Drive\\Overlijdensdata Curaçao 1831-1950\\GPT output\\batch_tGzEVvJnzgnMZYELmDM11OBY_output.jsonl', f'g:\\My Drive\\Overlijdensdata Curaçao 1831-1950\\GPT output\\batch_SuIkaQMj2bjWoRU59GNGS8yL_output.jsonl']

In [3]:
rows = []
for file in files:
    with jsonlines.open(file, mode='r') as reader:
        for line in reader:
            id = line['custom_id']
            response = line['response']['body']['choices'][0]['message']['content'].strip()
            answer = ast.literal_eval(response)
            rows.append({"ID":id, "answer":answer})

In [4]:
rows

[{'ID': 'O.R. 1879 Buiten 2e distr. 001',
  'answer': {'STILLBORN_CERTIFICATE': 'False',
   'CERTIFICATE_DATE': '01-01-1879',
   'NAME_INFORMANT_1': 'Theodorus Martes',
   'PROFESSION_INFORMANT_1': 'schoenmaker',
   'AGE_INFORMANT_1': '46',
   'NAME_INFORMANT_2': 'Felix zeven',
   'PROFESSION_INFORMANT_2': 'metselaar',
   'AGE_INFORMANT_2': '41',
   'DATE_OF_DEATH': '31-12-1878',
   'TIME_OF_DEATH': '04:00',
   'NAME': 'Cabrula Eliman',
   'AGE': '7 jaren, 9 maanden, 12 dagen',
   'PROFESSION': '#',
   'PLACE_OF_BIRTH': 'Curaçao',
   'DATE_OF_BIRTH': '18-03-1871',
   'SEX': 'v',
   'MARITAL_STATUS': '#',
   'FATHER_NAME': '#',
   'FATHER_DECEASED': '#',
   'FATHER_PROFESSION': '#',
   'MOTHER_NAME': 'Adelina Leenen',
   'MOTHER_DECEASED': 'in leven',
   'MOTHER_PROFESSION': 'waschver',
   'PARTNER_NAME': '#',
   'PARTNER_DECEASED': '#',
   'PARTNER_PROFESSION': '#'}},
 {'ID': 'O.R. 1879 Buiten 2e distr. 002',
  'answer': {'STILLBORN_CERTIFICATE': 'False',
   'CERTIFICATE_DATE': '06-01-

In [5]:
df = pd.json_normalize(rows)
df.columns = df.columns.str.replace("answer.","")
df

,ID,STILLBORN_CERTIFICATE,CERTIFICATE_DATE,NAME_INFORMANT_1,PROFESSION_INFORMANT_1,AGE_INFORMANT_1,NAME_INFORMANT_2,PROFESSION_INFORMANT_2,AGE_INFORMANT_2,DATE_OF_DEATH,...,MARITAL_STATUS,FATHER_NAME,FATHER_DECEASED,FATHER_PROFESSION,MOTHER_NAME,MOTHER_DECEASED,MOTHER_PROFESSION,PARTNER_NAME,PARTNER_DECEASED,PARTNER_PROFESSION
0,O.R. 1879 Buiten 2e distr. 001,False,01-01-1879,Theodorus Martes,schoenmaker,46,Felix zeven,metselaar,41,31-12-1878,...,#,#,#,#,Adelina Leenen,in leven,waschver,#,#,#
1,O.R. 1879 Buiten 2e distr. 002,False,06-01-1879,Theopole Maduro,timmerman,30,Joseph Anthony Benendt,planter,56,04-01-1879,...,#,#,#,#,Maraya Dendt,in leven,hoedenmaakster,#,#,#
2,O.R. 1879 Buiten 2e distr. 003,False,07-01-1879,Albertus Jonies,zes en Twintig,26,Cornelis Martis,veldarbeider,31,06-01-1879,...,#,#,#,#,Rita Ajarda,in leven,naaister,#,#,#
3,O.R. 1879 Buiten 2e distr. 004,False,19-01-1879,Felix Martis,veldarbeider,37,Cornelis William,metselaar,25,19-01-1879,...,#,Aurelio Maduro,in leven,#,Maria Elizabeth,in leven,#,#,#,#
4,O.R. 1879 Buiten 2e distr. 005,False,20-01-1879,"Eugene Adans, Ja Buento",veldarbeider,33,Narcillo Martien,veldarbeider,27,20-01-1879,...,#,onbekend,#,#,onbekend,#,#,#,#,#
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23934,O.R. 1949 Stad 633,False,29-12-1959,"Curiel, Sylverie",aanspreker,71,NaN,NaN,NaN,20-12-1959,...,gehuwd,"Angelardt, Antrict",overleden,#,"Winklaar, Maria Henriette",overleden,#,"Curilia, Margaretha Christiaan",#,#
23935,O.R. 1949 Stad 634,False,29-12-1959,Julio,aanspreker,59,NaN,NaN,NaN,28-12-1959,...,#,#,#,#,"Corguek, Margarita Adriana",overleden,#,#,#,#
23936,O.R. 1949 Stad 635,False,30-12-1959,Julio,aanspreker,59,NaN,NaN,NaN,17-12-1959,...,#,Priemed Nicolaas Martinus Rodewijk,overleden,#,"Emilia, Cathalina",overleden,#,#,#,#
23937,O.R. 1949 Stad 636,False,30-12-1959,"Curiel, Sylverie",aanspreker,71,NaN,NaN,NaN,24-12-1959,...,#,"Gomez, Rimond",#,#,"Gomez, Ruth",#,#,#,#,#


## Post-process entities

In [6]:
def get_cert_nr(id):
    cert_id = id.split(' ')[-1]
    return int(cert_id)

In [28]:
# add text data and certificate number
text_data = pd.read_excel(f'G:\My Drive\Overlijdensdata Curaçao 1831-1950\Transkribus HTR output\Transkribus HTR output.xlsx', index_col=0)
table = text_data.merge(df, on='ID').drop(columns=['TEXT'])
table.insert(2, 'CERTIFICATE_NR', 'temp')
table['CERTIFICATE_NR'] = table.apply(lambda x: get_cert_nr(x['ID']), axis=1)
table

,ID,TEXT_GPT,CERTIFICATE_NR,STILLBORN_CERTIFICATE,CERTIFICATE_DATE,NAME_INFORMANT_1,PROFESSION_INFORMANT_1,AGE_INFORMANT_1,NAME_INFORMANT_2,PROFESSION_INFORMANT_2,...,MARITAL_STATUS,FATHER_NAME,FATHER_DECEASED,FATHER_PROFESSION,MOTHER_NAME,MOTHER_DECEASED,MOTHER_PROFESSION,PARTNER_NAME,PARTNER_DECEASED,PARTNER_PROFESSION
0,O.R. 1879 Buiten 2e distr. 001,No. 1. Heden den Eersten Januari des Jaars Een...,1,False,01-01-1879,Theodorus Martes,schoenmaker,46,Felix zeven,metselaar,...,#,#,#,#,Adelina Leenen,in leven,waschver,#,#,#
1,O.R. 1879 Buiten 2e distr. 002,No. 2. Heden den zesden Januari des Jaars Een ...,2,False,06-01-1879,Theopole Maduro,timmerman,30,Joseph Anthony Benendt,planter,...,#,#,#,#,Maraya Dendt,in leven,hoedenmaakster,#,#,#
2,O.R. 1879 Buiten 2e distr. 003,No. 3. Heden den Zevenden January des Jaars Ee...,3,False,07-01-1879,Albertus Jonies,zes en Twintig,26,Cornelis Martis,veldarbeider,...,#,#,#,#,Rita Ajarda,in leven,naaister,#,#,#
3,O.R. 1879 Buiten 2e distr. 004,No. 6 Heden den Negentienden Januari des Jaars...,4,False,19-01-1879,Felix Martis,veldarbeider,37,Cornelis William,metselaar,...,#,Aurelio Maduro,in leven,#,Maria Elizabeth,in leven,#,#,#,#
4,O.R. 1879 Buiten 2e distr. 005,No. 5 Heden den Twintigsten Januari des Jaars ...,5,False,20-01-1879,"Eugene Adans, Ja Buento",veldarbeider,33,Narcillo Martien,veldarbeider,...,#,onbekend,#,#,onbekend,#,#,#,#,#
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23934,O.R. 1949 Stad 633,"Nr. 633 veertig Renvooi, zomede de doorhaling ...",633,False,29-12-1959,"Curiel, Sylverie",aanspreker,71,NaN,NaN,...,gehuwd,"Angelardt, Antrict",overleden,#,"Winklaar, Maria Henriette",overleden,#,"Curilia, Margaretha Christiaan",#,#
23935,O.R. 1949 Stad 634,"Nr. 634 veertig Renvooi, nemede de doorhaling ...",634,False,29-12-1959,Julio,aanspreker,59,NaN,NaN,...,#,#,#,#,"Corguek, Margarita Adriana",overleden,#,#,#,#
23936,O.R. 1949 Stad 635,"Nr. 635 veertig Renvooi, mede de door haling v...",635,False,30-12-1959,Julio,aanspreker,59,NaN,NaN,...,#,Priemed Nicolaas Martinus Rodewijk,overleden,#,"Emilia, Cathalina",overleden,#,#,#,#
23937,O.R. 1949 Stad 636,Nr. 636 veertig Renvooi zomede de doorhaling v...,636,False,30-12-1959,"Curiel, Sylverie",aanspreker,71,NaN,NaN,...,#,"Gomez, Rimond",#,#,"Gomez, Ruth",#,#,#,#,#


In [16]:
def improve_cert_date(id, date):
    valid_date = regex.search(r'\d\d-\d\d-\d\d\d\d', date)
    if valid_date:
        return str(date)[:-4]+id[5:9]
    else:
        return date

In [ ]:
improved = table[['ID', 'CERTIFICATE_DATE']]
improved['CERTIFICATE_DATE_IMPROVED'] = improved.apply(lambda x: improve_cert_date(x['ID'], x['CERTIFICATE_DATE']) if x['CERTIFICATE_DATE'] is not np.nan else None, axis=1)
improved['same'] = (improved['CERTIFICATE_DATE']==improved['CERTIFICATE_DATE_IMPROVED']) | (improved['CERTIFICATE_DATE'].isna() & improved['CERTIFICATE_DATE_IMPROVED'].isna())
improved

In [30]:
table['CERTIFICATE_DATE'] = table.apply(lambda x: improve_cert_date(x['ID'], x['CERTIFICATE_DATE']) if x['CERTIFICATE_DATE'] is not np.nan else None, axis=1)

In [31]:
def improve_death_date(cert_date, date):
    valid_cert_date = regex.search(r'\d\d-\d\d-\d\d\d\d', cert_date)
    valid_date = regex.search(r'\d\d-\d\d-\d\d\d\d', date)
    if valid_cert_date and valid_date:
        cert_month = str(cert_date)[3:5]
        month = str(date)[3:5]
        cert_year = str(cert_date)[-4:]
        year = str(date)[-4:]
        # set year to cert_year-1 if month later than cert_month
        if int(month)>int(cert_month):
            return str(date)[:-4]+str(int(cert_year)-1)
        # set year to cert_year
        else:
            return str(date)[:-4]+cert_year
    return date

In [32]:
print(improve_death_date('06-01-1879','04-01-1879'))
print(improve_death_date('30-12-1949','15-11-1959'))

04-01-1879
15-11-1949


In [ ]:
improved = table[['ID','CERTIFICATE_DATE', 'DATE_OF_DEATH']]
improved['IMPROVED'] = improved.apply(lambda x: improve_death_date(x['CERTIFICATE_DATE'], x['DATE_OF_DEATH']) if x['DATE_OF_DEATH'] is not np.nan else None, axis=1)
improved['same'] = (improved['DATE_OF_DEATH']==improved['IMPROVED']) | (improved['DATE_OF_DEATH'].isna() & improved['IMPROVED'].isna())
improved

In [34]:
table['DATE_OF_DEATH'] = table.apply(lambda x: improve_death_date(x['CERTIFICATE_DATE'], x['DATE_OF_DEATH']) if x['DATE_OF_DEATH'] is not np.nan else None, axis=1)

In [36]:
table

,ID,TEXT_GPT,CERTIFICATE_NR,STILLBORN_CERTIFICATE,CERTIFICATE_DATE,NAME_INFORMANT_1,PROFESSION_INFORMANT_1,AGE_INFORMANT_1,NAME_INFORMANT_2,PROFESSION_INFORMANT_2,...,MARITAL_STATUS,FATHER_NAME,FATHER_DECEASED,FATHER_PROFESSION,MOTHER_NAME,MOTHER_DECEASED,MOTHER_PROFESSION,PARTNER_NAME,PARTNER_DECEASED,PARTNER_PROFESSION
0,O.R. 1879 Buiten 2e distr. 001,No. 1. Heden den Eersten Januari des Jaars Een...,1,False,01-01-1879,Theodorus Martes,schoenmaker,46,Felix zeven,metselaar,...,#,#,#,#,Adelina Leenen,in leven,waschver,#,#,#
1,O.R. 1879 Buiten 2e distr. 002,No. 2. Heden den zesden Januari des Jaars Een ...,2,False,06-01-1879,Theopole Maduro,timmerman,30,Joseph Anthony Benendt,planter,...,#,#,#,#,Maraya Dendt,in leven,hoedenmaakster,#,#,#
2,O.R. 1879 Buiten 2e distr. 003,No. 3. Heden den Zevenden January des Jaars Ee...,3,False,07-01-1879,Albertus Jonies,zes en Twintig,26,Cornelis Martis,veldarbeider,...,#,#,#,#,Rita Ajarda,in leven,naaister,#,#,#
3,O.R. 1879 Buiten 2e distr. 004,No. 6 Heden den Negentienden Januari des Jaars...,4,False,19-01-1879,Felix Martis,veldarbeider,37,Cornelis William,metselaar,...,#,Aurelio Maduro,in leven,#,Maria Elizabeth,in leven,#,#,#,#
4,O.R. 1879 Buiten 2e distr. 005,No. 5 Heden den Twintigsten Januari des Jaars ...,5,False,20-01-1879,"Eugene Adans, Ja Buento",veldarbeider,33,Narcillo Martien,veldarbeider,...,#,onbekend,#,#,onbekend,#,#,#,#,#
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23934,O.R. 1949 Stad 633,"Nr. 633 veertig Renvooi, zomede de doorhaling ...",633,False,29-12-1949,"Curiel, Sylverie",aanspreker,71,NaN,NaN,...,gehuwd,"Angelardt, Antrict",overleden,#,"Winklaar, Maria Henriette",overleden,#,"Curilia, Margaretha Christiaan",#,#
23935,O.R. 1949 Stad 634,"Nr. 634 veertig Renvooi, nemede de doorhaling ...",634,False,29-12-1949,Julio,aanspreker,59,NaN,NaN,...,#,#,#,#,"Corguek, Margarita Adriana",overleden,#,#,#,#
23936,O.R. 1949 Stad 635,"Nr. 635 veertig Renvooi, mede de door haling v...",635,False,30-12-1949,Julio,aanspreker,59,NaN,NaN,...,#,Priemed Nicolaas Martinus Rodewijk,overleden,#,"Emilia, Cathalina",overleden,#,#,#,#
23937,O.R. 1949 Stad 636,Nr. 636 veertig Renvooi zomede de doorhaling v...,636,False,30-12-1949,"Curiel, Sylverie",aanspreker,71,NaN,NaN,...,#,"Gomez, Rimond",#,#,"Gomez, Ruth",#,#,#,#,#


In [37]:
# export post-processed table
table.to_excel(f'g:\\My Drive\\Overlijdensdata Curaçao 1831-1950\\GPT output\\Overlijdensdata_GPT.xlsx')